## Homework 4, Problem 4 Classification on real data

ECE C143A/C243A, Spring Quarter 2018, Prof. J.C. Kao, TAs T. Monsoor, X. Jiang and X. Yang

## Background
Neural prosthetic systems can be built based on classifying neural activity related to planning. As described in class, this is analogous to mapping patterns of neural activity to keys on a keyboard.
In this problem, we will apply the results of Problems 1 and 2 to real neural data. The neural data were recorded using a 100-electrode array in premotor cortex of a macaque monkey1. The dataset can be found on CCLE as `ps4_realdata.mat`.

The following describes the data format. The `.mat` file is loaded into Python as a dictionary with two keys: `train_trial` contains the training data and `test_trial` contains the test data. Each of these contains spike trains recorded simultaneously from 97 neurons while the monkey reached 91 times along each of 8 different reaching angles.

The spike train recorded from the $i_{th}$ neuron on the $n_{th}$ trial of the $k_{th}$ reaching angle is accessed as 

`data['train_trial'][n,k][1][i,:]`

where n = 0,...,90, k = 0,...,7, and i = 0, . . . , 96.  The [1] in between [n,k] and [i,:] does not mean anything for this assignment and is simply an "artifact" of how the data is structured. A spike train is represented as a sequence of zeros and ones, where time is discretized in 1 ms steps. A zero indicates that the neuron did not spike in the 1 ms bin, whereas a one indicates that the neuron spiked once in the 1 ms bin. The structure test trial has the same format as train trial.

Each spike train is 700 ms long (and thus represented by an array of length 700).  This comprises a 200ms baseline period (before the reach target turned on), a 500ms planning period (after the reach target turned on).  Because it takes time for information about the reach target to arrive in premotor cortex (due to the time required for action potentials to propagate and for visual processing), we will ignore the first 150ms of the planning period.  *** FOR THIS PROBLEM, we will take spike counts for each neuron within a single 200ms bin starting 150ms after the reach target turns on. ***

In other words, to calculate firing rates, you will calculate it over the 200ms window: 

`data['train_trial'][n,k][1][i,350:550]`

In [17]:
from __future__ import division
from scipy.misc import factorial
import numpy as np
import numpy.matlib as npm
import matplotlib.pyplot as plt
import scipy.special
import scipy.io as sio
import math

data = sio.loadmat('ps4_realdata.mat') # load the .mat file.
NumTrainData = data['train_trial'].shape[0]
NumClass = data['train_trial'].shape[1]
NumTestData = data['test_trial'].shape[0]

# Reloading any code written in external .py files.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### (a) (8 points) 
Fit the ML parameters of model i) to the training data (91 × 8 observations of a length 97 array of neuron firing rates). 

To calculate the firing rates, use a single 200ms bin starting from 150ms after the target turns on.  This corresponds to using `data['train_trial'][n,k][1][i, 350:550]` to calculate all firing rates.  This corresponds to a 200ms window that turns on 150ms after the reach turns on.

Then, use these parameters to classify the test data (91 × 8 data points) according to the decision rule (1). What is the percent of test data points correctly classified?

In [33]:
##4a

# Calculate the firing rates.

trainDataArr =  np.zeros((NumClass,NumTrainData,97)) # contains the firing rates for all neurons on all 8 x 91 trials in the training set
testDataArr =  np.zeros((NumClass,NumTestData,97)) # for the testing set.

for classIX in range(NumClass):
    for trainDataIX in range(NumTrainData):   
        trainDataArr[classIX,trainDataIX,:] = np.sum(data['train_trial'][trainDataIX,classIX][1][:,350:550],1)
    for testDataIX in range(NumTestData):        
        testDataArr[classIX,testDataIX,:]=np.sum(data['test_trial'][testDataIX,classIX][1][:,350:550],1)
#====================================================#
# YOUR CODE HERE:
#   Fit the ML parameters of model i) to training data
#====================================================#
NumFea = np.shape ( trainDataArr)
actLabel= np.matlib.repmat(np.arange(NumClass).reshape(-1, 1),1,NumTestData)

modParam= {}
modParam['pi'] = 1.0/8.0
modParam['mean'] = np.zeros([8,97])

modParam['cov'] = np.zeros([8,97,97])
for classIX in range(NumClass):
    modParam['mean'][classIX] = np.mean(trainDataArr[classIX],axis=0)
    modParam['cov'][classIX] = np.cov(trainDataArr[classIX].T)
Tcovariance=np.sum(modParam['cov'],axis=0)/8

pred = []
for testclassIX in range(NumClass):
    for testDataIX in range(NumTestData):
        dec = []
        for classIX in range(NumClass):
            mean1 = modParam['mean'][classIX]
            Z = np.exp(-(np.matmul(np.matmul(testDataArr[testclassIX,testDataIX]-mean1,np.linalg.inv(Tcovariance)),\
                (testDataArr[testclassIX,testDataIX]-mean1).T))/2)/(np.sqrt(np.linalg.det(Tcovariance))*((2*np.pi)**(97/2)))
            dec.append(Z)
        
        if np.argmax(dec)==testclassIX:
            pred.append(1)
        else:
            pred.append(0)

print(sum(pred)/len(pred))


#====================================================#
# YOUR CODE HERE:
#   Classify the test data and print the accuracy
#====================================================#

#====================================================#
# END YOUR CODE
#====================================================# 


0.960164835165


#### Question:
What is the percent of test data points correctly classified?

#### Your answer: 
0.960164835165

### (b) (6 points) 
Repeat part (a) for model ii). You `should encounter a Python error` when classifying the test data. What is this error? Why did the Python error occur? What would we need to do to correct this error?

To be concrete, the output of this cell should be a `Python error` and that's all fine.  But we want you to understand what the error is so we can fix it later.


In [35]:
##4b

#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters of model ii) to training data
#====================================================#
pred = []
for testclassIX in range(NumClass):
    for testDataIX in range(NumTestData):
        dec = []
        for classIX in range(NumClass):
            mean1 = modParam1['mean'][classIX]
            co = modParam['cov'][classIX]
            Z = np.exp(-(np.matmul(np.matmul(testDataArr[testclassIX,testDataIX]-mean1,np.linalg.pinv(co)),\
                (testDataArr[testclassIX,testDataIX]-mean1).T))/2)/(np.sqrt(np.linalg.det(co))*((2*np.pi)**(97/2)))
            dec.append(Z)
        
        if np.argmax(dec)==testclassIX:
            pred.append(1)
        else:
            pred.append(0)

print(sum(pred)/len(pred))
#====================================================#
# END YOUR CODE
#====================================================# 


C:\ProgramData\Anaconda2\envs\py2\lib\site-packages\ipykernel\__main__.py:14: RuntimeWarning: divide by zero encountered in double_scalars
C:\ProgramData\Anaconda2\envs\py2\lib\site-packages\ipykernel\__main__.py:14: RuntimeWarning: invalid value encountered in double_scalars


0.0782967032967


#### Question:
Why did the python error occur? What would we need to do to correct this error?

#### Your answer:  
Covariance matrix was singular for some class. So I used the pseudo inverse method np.linalg.pinv.But the determinant of singular matrix is zero,which can not be dominator. It might be helpful if we delete some bad data to make the covariance invertable.

### (c) (8 points) 
Correct the problem from part (b) by detecting and then removing offending neurons that cause the error. Now, what is the percent of test data points correctly classified? Is it higher or lower than your answer to part (a)? Why might this be?

In [68]:
a=np.sum(trainDataArr,axis=1)
#print(np.where(a > 0.0)[1])
offending=[]
for classIX in range(NumClass):
    x = np.sum(trainDataArr[classIX],axis=0)
    offending=np.append(offending,np.where(x == 0.0))
offending=np.unique(offending)
keep=np.arange(97)
keep=np.delete(keep,offending)
print(np.shape(keep))
#     trainArr = trainDataArr[classIX,:,keep]
#     testArr = testDataArr[classIX,:,keep]


(87L,)


C:\ProgramData\Anaconda2\envs\py2\lib\site-packages\ipykernel\__main__.py:9: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


In [73]:
##4c
neuronsToRemove = []
#====================================================#
# YOUR CODE HERE:
#   Detect and then remove the offending neurons, so that 
#   you no longer run into the bug in part (b).
#====================================================#
#find affending neurons
offending=[]
for classIX in range(NumClass):
    x = np.sum(trainDataArr[classIX],axis=0)
    offending=np.append(offending,np.where(x == 0.0))
offending=np.unique(offending)
keep=np.arange(97)
keep=np.delete(keep,offending)


train = trainDataArr[:,:,keep]
test = testDataArr[:,:,keep]


mean2 = np.zeros([8,87])
covariance2 = np.zeros([8,87,87])
for classIX in range(NumClass):
    mean2[classIX] = np.mean(train[classIX],axis=0)
    covariance2[classIX] = np.cov(train[classIX].T)
    

pred = []
for testclassIX in range(NumClass):
    for testDataIX in range(NumTestData):
        dec = []
        for classIX in range(NumClass):
            mean =  mean2[classIX]
            co =  covariance2[classIX]
            t = test[testclassIX,testDataIX]-mean
            Z = np.matmul(np.matmul(t,np.linalg.inv(co)),t.T)
            Z = -Z/2-np.log(np.linalg.det(co))/2
            dec.append(Z)
        
        if np.argmax(dec)==testclassIX:
            pred.append(1)
        else:
            pred.append(0)

print(sum(pred)/len(pred))





#====================================================#
# END YOUR CODE
#====================================================# 
##
#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters,classify the test data and print the accuracy
#====================================================#

#====================================================#
# END YOUR CODE
#====================================================# 

C:\ProgramData\Anaconda2\envs\py2\lib\site-packages\ipykernel\__main__.py:15: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


0.440934065934


#### Question:
What is the percent of test data points correctly classified? Is it higher or lower than your answer to part (a)? Why might this be?

#### Your answer:  
0.44. It's lower than the result in part (a). The data for training model in each class is less than those in part (a). 
Model2 use covariance for each class, noise might affect the model parameter

### (d) (8 points) 
Now we classify using a naive Bayes model. Repeat part (a) for model iii). Keep the convention in part (c), where offending neurons were removed from the anal- ysis.

In [74]:
##4d
#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters,classify the test data and print the accuracy
#====================================================#


pred = []
for testclassIX in range(NumClass):
    for testDataIX in range(NumTestData):
        dec = []
        for classIX in range(NumClass):
            mean3 =  mean2[classIX]
            
            Z = 1
            for neuronIX in range(87):
                Z =Z * mean3[neuronIX]**test[testclassIX,testDataIX][neuronIX]*np.exp(-mean3[neuronIX])\
                /factorial(test[testclassIX,testDataIX][neuronIX])
            dec.append(Z)
        
        if np.argmax(dec)==testclassIX:
            pred.append(1)
        else:
            pred.append(0)

print(sum(pred)/len(pred))




#====================================================#
# END YOUR CODE
#====================================================# 

C:\ProgramData\Anaconda2\envs\py2\lib\site-packages\ipykernel\__main__.py:17: DeprecationWarning: `factorial` is deprecated!
Importing `factorial` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.factorial` instead.


0.92032967033


#### Question:
what is the percent of test data points correctly classified? 

#### Your answer: 
0.92032967033